In [1]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [2]:
!pip install -U torch torchvision torchaudio
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/
#超重要！！！！！
#ローカルPCでdatasetをdataset.zipに圧縮してMyDriveへコピーしてください。

# ZIPファイルをGoogle DriveからColabへコピー
!rsync -ah --progress /content/drive/MyDrive/dataset.zip /content/

# ZIPファイルを解凍
!unzip -o /content/dataset.zip -d /content/

# (任意) 解凍後のZIPファイルを削除してディスクを節約
!rm /content/dataset.zip

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: /content/dataset/images/train/2eff9c34-Ch120250118105049_learning_learning.jpg  
  inflating: /content/dataset/images/train/2f003409-Ch120250202102810.jpg  
  inflating: /content/dataset/images/train/2f049f65-Ch120240927165458.png  
  inflating: /content/dataset/images/train/2f0bc4ed-Ch120250206105059.jpg  
  inflating: /content/dataset/images/train/2f3a6e29-Ch120250130094245.jpg  
  inflating: /content/dataset/images/train/2f718430-Ch120241203103119.png  
  inflating: /content/dataset/images/train/2f860d3c-Ch120241225114440.png  
  inflating: /content/dataset/images/train/2f8d3420-Ch120241225114417.png  
  inflating: /content/dataset/images/train/2f93dd9e-Ch120240927115843.png  
  inflating: /content/dataset/images/train/2fa37910-Ch120250111161541.png  
  inflating: /content/dataset/images/train/2fac2083-Ch120241021101059.png  
  inflating: /content/dataset/images/train/2feafecb-Ch120250111161854.png  
  inflating: /content/dataset/images/t

In [5]:
model_path = "/content/drive/MyDrive/yolov8s.pt"
# YAMLファイルのパス
data_yaml = "/content/dataset/data.yaml"

In [6]:
import yaml


# YAMLファイルを読み込む
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

data.yaml のパスを更新しました。


In [7]:
import os
from ultralytics import YOLO
import numpy as np


def train_yolov8(model_path, data_yaml, epochs, batch_size, img_size, output_dir, iou_threshold, mosaic, mixup):
    """
    Train YOLOv8 model with enhanced settings for small overlapping object detection.
    """
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    # Train the model with optimized settings
    print(f"Training YOLOv8 with {epochs} epochs...")
    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project=output_dir,
        name="yolov8_training",
        iou=iou_threshold,  # Adjust IoU threshold for better object separation
        mosaic=mosaic,  # Enhance data augmentation with mosaic images
        mixup=mixup,  # Mixup augmentation for diverse training
        degrees=180,  # Rotation augmentation
    )

    print(f"Training complete. Results saved to {output_dir}")

# Colab 用にパラメータを手動で指定
model_path = "/content/drive/MyDrive/yolov8s.pt"  # モデルのパス (Google Drive に保存している場合は適宜変更)
#data_yaml = "/content/drive/MyDrive/MaiNDLab/dataset/data.yaml"  # データセットの YAML ファイル
epochs = 200  # エポック数
batch_size = 64  # バッチサイズ
img_size = 1280  # 画像サイズ
iou_threshold = 0.3  # IoU 閾値
mosaic = 1.0  # モザイクの確率
mixup = 0.2  # Mixup の確率
output_dir = "runs_experiment"  # 出力ディレクトリ

# 出力ディレクトリを作成
os.makedirs(output_dir, exist_ok=True)

# YOLOv8 の学習実行
train_yolov8(
    model_path=model_path,
    data_yaml=data_yaml,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size,
    output_dir=output_dir,
    iou_threshold=iou_threshold,
    mosaic=mosaic,
    mixup=mixup
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Loading model from /content/drive/MyDrive/yolov8s.pt
Training YOLOv8 with 200 epochs...
Ultralytics 8.3.88 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/yolov8s.pt, data=/content/dataset/data.yaml, epochs=200, time=None, patience=100, batch=64, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs_experiment, name=yolov8_training, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, mult

100%|██████████| 755k/755k [00:00<00:00, 14.8MB/s]


Overriding model.yaml nc=80 with nc=11

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 65.2MB/s]


AMP: checks passed ✅


...

    200/200      34.4G     0.7823     0.3542      1.056       1013        960: 100%|██████████| 31/31 [00:16<00:00,  1.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.46it/s]

                   all        563      11164      0.978      0.913      0.941      0.708



200 epochs completed in 1.499 hours.
Optimizer stripped from runs_experiment/yolov8_training/weights/last.pt, 22.6MB
Optimizer stripped from runs_experiment/yolov8_training/weights/best.pt, 22.6MB

Validating runs_experiment/yolov8_training/weights/best.pt...
Ultralytics 8.3.88 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,129,841 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:12<00:00,  2.42s/it]


                   all        563      11164      0.979      0.914      0.944      0.713
                  Huki        139        852      0.952      0.982      0.961      0.681
                Inngen        191       2070      0.973      0.741      0.851      0.634
               Kabocha        287       1794      0.971      0.968      0.979      0.638
              Ninnjinn        382       2353      0.974      0.974      0.983      0.723
              Shiitake        331       2059      0.984      0.988      0.991      0.772
             Gochujang        179        555      0.984      0.901      0.933      0.743
              F_renkon         44        269      0.997      0.981      0.989      0.867
                daikon         51        323      0.996      0.991      0.995      0.807
                 ganmo         44        266      0.987      0.981      0.988       0.78
                 gobou         51        308      0.979      0.818      0.896      0.611
              konnyak